## Imports

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os
import glob
import csv

### --- 1. INITIALIZE MEDIAPIPE ---

In [ ]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=True, # We are processing static images
    max_num_hands=1,
    min_detection_confidence=0.5
)

I0000 00:00:1762874670.824048  978160 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1762874670.873263  978268 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.0.7-0ubuntu0.24.04.2), renderer: llvmpipe (LLVM 20.1.2, 256 bits)


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


### --- 2. DEFINE CLASSES AND DATA FOLDER ---

In [3]:
DATA_DIR = 'asl_alphabet_train'
class_names = [
    'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 
    'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 
    'del', 'nothing', 'space'
]

In [5]:
# --- 3. CREATE THE NEW CSV FILE ---
csv_file_name = 'asl_landmarks_v2.csv' # New name
print(f"Opening {csv_file_name} to write...")

# A 1D vector of 63 zeros. This is our "nothing" feature.
ZERO_VECTOR = [0.0] * 63 

with open(csv_file_name, mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    
    # --- 4. WRITE THE HEADER ROW ---
    header = ['label']
    for i in range(21): # 21 landmarks
        header.extend([f'x{i}', f'y{i}', f'z{i}'])
    csv_writer.writerow(header)

    # --- 5. LOOP THROUGH ALL IMAGES ---
    for label in class_names:
        folder_path = os.path.join(DATA_DIR, label, '*.jpg')
        image_files = glob.glob(folder_path)
        print(f"Processing {len(image_files)} images for class: {label}")
        
        for image_path in image_files:
            
            # --- !!! NEW LOGIC !!! ---
            # If the class is "nothing", don't even run MediaPipe.
            # Just write the zero vector.
            if label == 'nothing':
                csv_writer.writerow([label] + ZERO_VECTOR)
                continue # Go to the next image
            # --- END NEW LOGIC ---

            image = cv2.imread(image_path)
            if image is None:
                continue
                
            rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = hands.process(rgb_image)

            # --- 6. EXTRACT & NORMALIZE LANDMARKS ---
            if results.multi_hand_landmarks:
                hand_landmarks = results.multi_hand_landmarks[0]
                
                wrist = hand_landmarks.landmark[0]
                landmark_vector = []
                for landmark in hand_landmarks.landmark:
                    landmark_vector.append(landmark.x - wrist.x)
                    landmark_vector.append(landmark.y - wrist.y)
                    landmark_vector.append(landmark.z - wrist.z)
                    
                csv_writer.writerow([label] + landmark_vector)
            
            else:
                # --- !!! NEW LOGIC FOR FISTS !!! ---
                # If MediaPipe *fails* to find a hand for "del" or "space"
                # (which are fists), we will ALSO treat this as a zero vector.
                if label in ['del', 'space']:
                    csv_writer.writerow([label] + ZERO_VECTOR)
                # For all other classes ('A', 'B', etc.), if MediaPipe
                # fails, we skip it as "bad data".
                pass

print(f"--- Dataset Creation Complete! ---")
print(f"New dataset saved as: {csv_file_name}")
hands.close()

Opening asl_landmarks_v2.csv to write...
Processing 3000 images for class: A
Processing 3000 images for class: B
Processing 3000 images for class: C
Processing 3000 images for class: D
Processing 3000 images for class: E
Processing 3000 images for class: F
Processing 3000 images for class: G
Processing 3000 images for class: H
Processing 3000 images for class: I
Processing 3000 images for class: J
Processing 3000 images for class: K
Processing 3000 images for class: L
Processing 3000 images for class: M
Processing 3000 images for class: N
Processing 3000 images for class: O
Processing 3000 images for class: P
Processing 3000 images for class: Q
Processing 3000 images for class: R
Processing 3000 images for class: S
Processing 3000 images for class: T
Processing 3000 images for class: U
Processing 3000 images for class: V
Processing 3000 images for class: W
Processing 3000 images for class: X
Processing 3000 images for class: Y
Processing 3000 images for class: Z
Processing 3000 images 